In [2]:
pwd

'/home/hllqk/projects/yolo-mecd/ultralytics/train'

In [ ]:
# 划分训练集和验证集
import json
import os
from sklearn.model_selection import train_test_split

# 路径配置
dataset_root = "../datasets/dataverse_files"
train_json = os.path.join(dataset_root, "CitDet-train/train/train_annotations.json")
test_json = os.path.join(dataset_root, "CitDet-test/test/test_annotations.json")

# 创建验证集目录
os.makedirs(os.path.join(dataset_root, "CitDet-val"), exist_ok=True)

# 加载原始训练集标注
with open(train_json) as f:
    train_data = json.load(f)

image_ids = [img["id"] for img in train_data["images"]]
# 将train_data拆分为训练集和验证集按8:2比例划分
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

def filter_coco(data, keep_ids):
    """过滤COCO数据集中指定image_id的条目"""
    return {
        "images": [img for img in data["images"] if img["id"] in keep_ids],
        "annotations": [ann for ann in data["annotations"] if ann["image_id"] in keep_ids],
        "categories": data["categories"]
    }

# 保存新标注文件
with open(os.path.join(dataset_root, "CitDet-train/train_split.json"), "w") as f:
    json.dump(filter_coco(train_data, train_ids), f)

with open(os.path.join(dataset_root, "CitDet-val/val_split.json"), "w") as f:
    json.dump(filter_coco(train_data, val_ids), f)

print(f"划分完成：\n"
      f"- 新训练集: {len(train_ids)}张\n"
      f"- 验证集: {len(val_ids)}张\n"
      f"- 原测试集: 119张（保持不变）")

划分完成：
- 新训练集: 368张
- 验证集: 92张
- 原测试集: 119张（保持不变）


In [8]:
with open(os.path.join(dataset_root, "CitDet-train/train_split.json"), "r") as f:
    coco_data = json.load(f)
coco_categories = coco_data["categories"]
print(coco_categories)


[{'id': 0, 'name': 'Fruit-Citrus-0GcP', 'supercategory': 'none'}, {'id': 1, 'name': 'Fruit on Ground', 'supercategory': 'Fruit-Citrus-0GcP'}, {'id': 2, 'name': 'Fruit on Tree', 'supercategory': 'Fruit-Citrus-0GcP'}]


In [ ]:
# Train a detection model on the COCO dataset
# location=